# A. Informationen zum Notebook

<p>Herausgeber:  Tobias Vetterle, André Förster, Anna Hinzen, Fachstelle kommunales Bildungsmonitoring (KOSMO)</p>
<p>Version: 3.0.2 (23.05.2024)<p/>

<p>Dieses Notebook dient dazu, Gemeindedaten der Bundesagentur fuer Arbeit (BA) mit Bevoelkerungsdaten des Statistischen Bundesamtes (Destatis) aus den Jahren 2014 bis 2021 zu verknuepfen, um daraus die Entwicklung gemeindespezifischer SGB II-Quoten berechnen und klassifiziert auf einem Dashboard abbilden zu koennen. Hierbei wird auf folgende Originaldaten der BA und von Destatis zugegriffen, die von den entsprechenden Stellen bezogen werden koennen: </p>

- Statistik der Bundesagentur fuer Arbeit, Auftragsnummer 338634, Teil 1-5 (Eckwerte der Grundsicherung nach Gemeinden, Gebietsstand Dezember des jeweiligen Jahres, Jahresdurchschnittswerte, Datenstand: Maerz 2023, Daten zu Leistungen nach dem SGB II nach einer Wartezeit von 3 Monaten)
- Statistisches Bundesamt (Destatis), Bevoelkerungszahlen auf Gemeindeebene nach Geschlecht und Alter (Fortschreibung Zensus 2011, Stichtagsdaten zum 31.12. fuer Gebietsstand und Bevoelkerungszahl des jeweiligen Berichtsjahres (2014-2021))

Um die Aufbereitung der Daten korrekt durchfuehren zu koennen, muessen diese Notebook-Datei (.ipynb) sowie die Originaldaten bzw. -dateien der BA und von Destatis in einem lokalen Ordner gespeichert werden.

# 0. Vorbereitung der Daten

In [1]:
# Benoetigte Pakete importieren
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import locale

# Hinweis: die folgende Bibliothek muss manuell installiert werden
from mapclassify import NaturalBreaks
from mapclassify import FisherJenksSampled
from mapclassify import FisherJenks

# Warnmeldungen unterdruecken
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'mapclassify'

In [ ]:
# Begrenzung der Spaltendarstellung deaktivieren
pd.set_option("display.max_columns", None)

In [ ]:
# Einlesen der Gemeindedaten der Bundesagentur fuer Arbeit (BA) aus den Jahren 2014 bis 2021
# Dateinamen und Tabellennamen abrufen
file_names = [
    ("230328_EA_338634_Gemeindedaten_Teil1.xlsx", "2014"),
    ("230328_EA_338634_Gemeindedaten_Teil2.xlsx", "2015"),
    ("230328_EA_338634_Gemeindedaten_Teil2.xlsx", "2016"),
    ("230328_EA_338634_Gemeindedaten_Teil3.xlsx", "2017"),
    ("230328_EA_338634_Gemeindedaten_Teil3.xlsx", "2018"),
    ("230328_EA_338634_Gemeindedaten_Teil4.xlsx", "2019"),
    ("230328_EA_338634_Gemeindedaten_Teil4.xlsx", "2020"),
    ("230328_EA_338634_Gemeindedaten_Teil5.xlsx", "2021")
]

# Dictionary fuer die Dataframes erstellen
dataframes = {}

# Ueber Dateien und Tabellennamen iterieren
for file_name, sheet_name in file_names:
    # Spezifisches Tabellenblatt in einen DataFrame einlesen
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    # DataFrame im Dictionary speichern
    dataframes["ba_" + sheet_name] = df

# Zugriff auf die Dataframes
ba_2014 = dataframes["ba_2014"]
ba_2015 = dataframes["ba_2015"]
ba_2016 = dataframes["ba_2016"]
ba_2017 = dataframes["ba_2017"]
ba_2018 = dataframes["ba_2018"]
ba_2019 = dataframes["ba_2019"]
ba_2020 = dataframes["ba_2020"]
ba_2021 = dataframes["ba_2021"]

In [ ]:
# Tabellenkopf bis zum Beginn des Datenbereichs in Zeile 15 anzeigen
ba_2021.head(15)

# 1. BA-Daten umformen

## 1.1 Grundlegende Umformung der Ausgangsdaten

In [ ]:
# Funktion zur Transformation der Daten definieren
def transform_dataframe(df):
    # Zeilen 0 bis 7 entfernen
    df = df.drop(df.index[0:7])

    # Letzte Zeile entfernen
    df = df.drop(df.index[-1])

    # Spalte 'Unnamed: 0' nach Leerzeichen trennen und in neue Spalten 'AGS' und 'Gemeinde' aufteilen
    df[['AGS', 'Gemeinde']] = df['Unnamed: 0'].str.split(' ', 1, expand=True)

    # Spalte 'Unnamed: 0' entfernen
    df = df.drop(['Unnamed: 0'], axis=1)

    # Erstellen einer Kontrollspalte 'Gesamtwerte_Kontrolle' mit den Spalten 'AGS', 'Gemeinde' und 'Unnamed: 1'
    Gesamtwerte_Kontrolle = df[['AGS', 'Gemeinde', 'Unnamed: 1']].copy()

    # Spalte 'Unnamed: 1' entfernen
    df.drop(['Unnamed: 1'], axis=1, inplace=True)

    # Nur Spalten ab der 23. Spalte behalten
    df = df.iloc[:, 23:]

    # Fehlende Werte in Zeile 5 mit den Werten der vorherigen Zeilen fuellen
    df.iloc[5] = df.iloc[5].fillna(df.iloc[4])
    df.iloc[5] = df.iloc[5].fillna(df.iloc[3])
    df.iloc[5] = df.iloc[5].fillna(df.iloc[2])

    # Zeilen 7, 9, 10, 11 und 13 entfernen
    df = df.drop([7, 9, 10, 11, 13])

    # Letzte beiden Werte in Zeile 1 mit den Werten der letzten beiden Spaltennamen fuellen
    df.iloc[1, -1] = df.columns[-1]
    df.iloc[1, -2] = df.columns[-2]

    # Neue Spaltennamen aus Zeile 1 erstellen
    new_columns = df.iloc[1].tolist()

    # Spaltennamen aktualisieren
    df = df.set_axis(new_columns, axis=1, inplace=False)

    # Zeile 12 entfernen
    df = df.drop(12)

    # Transformiertes DataFrame zurueckgeben
    return df

# Definierte Funktion auf die Daten anwenden und damit Dataframes erstellen
ba_2014 = transform_dataframe(ba_2014)
ba_2015 = transform_dataframe(ba_2015)
ba_2016 = transform_dataframe(ba_2016)
ba_2017 = transform_dataframe(ba_2017)
ba_2018 = transform_dataframe(ba_2018)
ba_2019 = transform_dataframe(ba_2019)
ba_2020 = transform_dataframe(ba_2020)
ba_2021 = transform_dataframe(ba_2021)

In [ ]:
# Tabellenkopf nach der Umformung anzeigen
ba_2021.head(2)

## 1.2 Spalte fuer Geschlecht ergaenzen und Tabelle entsprechend stapeln

In [ ]:
# Funktion zur Transformation der Daten definieren
def apply_transformation(df):
    # Daten (maennlich) aus den ersten 24 Spalten und den Spalten 48 und 49 extrahieren
    df_m = df.iloc[:, list(range(24)) + [48, 49]]

    # Daten (weiblich) aus den Spalten 24 bis 49 extrahieren
    df_w = df.iloc[:, 24:50]

    # Geschlecht fuer Daten (weiblich) auf "w" setzen
    df_w.loc[:, "Geschlecht"] = "w"

    # Geschlecht fuer Daten (maennlich) auf "m" setzen
    df_m.loc[:, "Geschlecht"] = "m"

    # Zeile 8 aus Daten (weiblich) entfernen
    df_w = df_w.drop([8])

    # Zeile 8 aus Daten (maennlich) entfernen
    df_m = df_m.drop([8])

    # Daten (maennlich und weiblich) zu einem DataFrame kombinieren
    combined_df = df_m.append(df_w, ignore_index=True)

    # Spalten umbenennen, indem Bindestriche "-" entfernt werden
    combined_df = combined_df.rename(columns=lambda x: x.replace('-', ''))

    return combined_df

# Definierte Funktion auf die Daten anwenden und damit Dataframes transformieren
ba_2014_combined = apply_transformation(ba_2014)
ba_2015_combined = apply_transformation(ba_2015)
ba_2016_combined = apply_transformation(ba_2016)
ba_2017_combined = apply_transformation(ba_2017)
ba_2018_combined = apply_transformation(ba_2018)
ba_2019_combined = apply_transformation(ba_2019)
ba_2020_combined = apply_transformation(ba_2020)
ba_2021_combined = apply_transformation(ba_2021)

In [ ]:
# Tabellenkopf nach der Umformung anzeigen
ba_2021_combined.head(1)

## 1.3 Gestapelte Zieltabellen erstellen

#### Die Tabelle enthaelt zum jetzigen Zeitpunkt Daten zu den Leistungsberechtigten (LB) nach Geschlecht und Altersgruppen, zu den arbeitsuchenden erwerbsfaehigen Leistungsberechtigten (ELB) nach Geschlecht und Schulabschluessen sowie zu den arbeitsuchenden ELB nach Geschlecht und Berufsausbildung. Wir teilen die Tabelle in drei separate Tabellen zu diesen Kennzahlen auf, um sie im weiteren Verlauf zielgerichtet bearbeiten zu koennen.

 - Ein Dataframe (DF) mit Leistungsberechtigten (LB) nach Geschlecht und Altersgruppen, das sich auf Anzahl der LB aggregieren laesst
     - Spalte "LB unter 15 Jahren" loeschen, da aus anderen Spalten summierbar
     - Spalte "LB unter 65 Jahren" loeschen, da aus anderen Spalten summierbar
     - Spalte "15 bis unter 18 Jahren (Kinder)" loeschen
 - Ein DF mit arbeitsuchenden erwerbsfaehigen Leistungsberechtigten (ELB) nach Geschlecht und Schulabschluessen (Unterscheidung Schulabschluesse nur fuer "arbeitsuchende ELB" moeglich)
     - Spalten 15 - 20 in der Ausgangsdatei
 - Ein DF mit arbeitsuchenden ELB nach Geschlecht und Berufsausbildung (Unterscheidung Berufsausbildung nur fuer "arbeitsuchende ELB" moeglich)
     - Spalten 21- 24 in der Ausgangsdatei

In [ ]:
# Funktion zur Transformation der Daten definieren
def apply_transformation(df, year):
    # relevante Spalten aus dem kombinierten Dataframe extrahieren
    LB_nach_Altersgruppen = df.iloc[:, [3, 4, 5, 6, 7, 8, 24, 25, 26]].copy()

    # Datensatz in langes Format umwandeln, indem Altersgruppen in separate Spalte transformiert werden
    LB_nach_Altersgruppen = pd.melt(LB_nach_Altersgruppen, id_vars=['AGS', 'Gemeinde', 'Geschlecht'], value_vars=['unter 3 Jahren', '3 bis unter 6 Jahren', '6 bis unter 15 Jahren', '15 bis unter 18 Jahren', '18 bis unter 25 Jahren', '25 bis unter 65 Jahren'], var_name='Altersgruppe', value_name='Anzahl')

    # Spalte hinzufuegen, um das Jahr zu speichern
    LB_nach_Altersgruppen['Jahr'] = year

    # Spalten "AGS" und "Jahr" in Ganzzahlen umwandeln
    LB_nach_Altersgruppen["AGS"] = pd.to_numeric(LB_nach_Altersgruppen["AGS"], downcast="integer")
    LB_nach_Altersgruppen["Jahr"] = pd.to_numeric(LB_nach_Altersgruppen["Jahr"], downcast="integer")

    # gleicher Vorgang fuer arbeitsuchende ELB nach Schulabschluessen
    ELB_arbeitsuchend_nach_Schulabschluessen = df.iloc[:, [13, 14, 15, 16, 17, 18, 19, 24, 25, 26]].copy()
    ELB_arbeitsuchend_nach_Schulabschluessen = pd.melt(ELB_arbeitsuchend_nach_Schulabschluessen, id_vars=['AGS', 'Gemeinde', 'Geschlecht', 'arbeitsuchende ELB insgesamt'], value_vars=['kein Hauptschulabschluss', 'Hauptschulabschluss', 'mittlere Reife', 'Fachhochschule', 'Abitur/ Hochschulreife', 'keine Angabe'], var_name='Schulabschluss', value_name='Anzahl')
    ELB_arbeitsuchend_nach_Schulabschluessen['Jahr'] = year
    ELB_arbeitsuchend_nach_Schulabschluessen["AGS"] = pd.to_numeric(ELB_arbeitsuchend_nach_Schulabschluessen["AGS"], downcast="integer")
    ELB_arbeitsuchend_nach_Schulabschluessen["Jahr"] = pd.to_numeric(ELB_arbeitsuchend_nach_Schulabschluessen["Jahr"], downcast="integer")

    # gleicher Vorgang fuer die ELB arbeitsuchend nach Berufsausbildung
    ELB_arbeitsuchend_nach_Berufsausbildung = df.iloc[:, [13, 20, 21, 22, 23, 24, 25, 26]].copy()
    ELB_arbeitsuchend_nach_Berufsausbildung = pd.melt(ELB_arbeitsuchend_nach_Berufsausbildung, id_vars=['AGS', 'Gemeinde', 'Geschlecht', 'arbeitsuchende ELB insgesamt'], value_vars=['ohne abgeschlossene Berufsausbildung', 'betriebliche/schulische Ausbildung', 'akademische Ausbildung', 'keine Angabe'], var_name='Berufsausbildung', value_name='Anzahl')
    ELB_arbeitsuchend_nach_Berufsausbildung['Jahr'] = year
    ELB_arbeitsuchend_nach_Berufsausbildung["AGS"] = pd.to_numeric(ELB_arbeitsuchend_nach_Berufsausbildung["AGS"], downcast="integer")
    ELB_arbeitsuchend_nach_Berufsausbildung["Jahr"] = pd.to_numeric(ELB_arbeitsuchend_nach_Berufsausbildung["Jahr"], downcast="integer")

    # Rueckgabe der Ergebnisse in separaten Dataframes
    return LB_nach_Altersgruppen, ELB_arbeitsuchend_nach_Schulabschluessen, ELB_arbeitsuchend_nach_Berufsausbildung

# Definierte Funktion auf die Daten anwenden und damit Dataframes transformieren
LB_nach_Altersgruppen_2014, ELB_arbeitsuchend_nach_Schulabschluessen_2014, ELB_arbeitsuchend_nach_Berufsausbildung_2014 = apply_transformation(ba_2014_combined, 2014)
LB_nach_Altersgruppen_2015, ELB_arbeitsuchend_nach_Schulabschluessen_2015, ELB_arbeitsuchend_nach_Berufsausbildung_2015 = apply_transformation(ba_2015_combined, 2015)
LB_nach_Altersgruppen_2016, ELB_arbeitsuchend_nach_Schulabschluessen_2016, ELB_arbeitsuchend_nach_Berufsausbildung_2016 = apply_transformation(ba_2016_combined, 2016)
LB_nach_Altersgruppen_2017, ELB_arbeitsuchend_nach_Schulabschluessen_2017, ELB_arbeitsuchend_nach_Berufsausbildung_2017 = apply_transformation(ba_2017_combined, 2017)
LB_nach_Altersgruppen_2018, ELB_arbeitsuchend_nach_Schulabschluessen_2018, ELB_arbeitsuchend_nach_Berufsausbildung_2018 = apply_transformation(ba_2018_combined, 2018)
LB_nach_Altersgruppen_2019, ELB_arbeitsuchend_nach_Schulabschluessen_2019, ELB_arbeitsuchend_nach_Berufsausbildung_2019 = apply_transformation(ba_2019_combined, 2019)
LB_nach_Altersgruppen_2020, ELB_arbeitsuchend_nach_Schulabschluessen_2020, ELB_arbeitsuchend_nach_Berufsausbildung_2020 = apply_transformation(ba_2020_combined, 2020)
LB_nach_Altersgruppen_2021, ELB_arbeitsuchend_nach_Schulabschluessen_2021, ELB_arbeitsuchend_nach_Berufsausbildung_2021 = apply_transformation(ba_2021_combined, 2021)

In [ ]:
# Tabellenkopf der Tabelle zu den Leistungsberechtigten anzeigen
LB_nach_Altersgruppen_2021.head(4)

# 2. Bevoelkerungsdaten umformen

#### Nachdem die Daten zu den LB nun in der gewuenschten Form vorliegen, wenden wir uns den Bevoelkerungsdaten zu und formen diese ebenfalls um, sodass sie im naechsten Schritt mit den Daten zu den LB verknuepft werden koennen.

In [ ]:
# Liste mit den Dateinamen erstellen
file_names = glob.glob('bevoelkerung_alter_geschlecht_*.csv')

# Bevoelkerungsdaten einlesen und umformen

# Schleife ueber alle Dateien
for file in file_names:
    # Jahreszahl aus dem Dateinamen extrahieren
    year = file.split('_')[-1].split('.')[0]

    # Daten einlesen und jede CSV in einem DataFrame mit der Bezeichnung "bevoelkerung_yyyy" speichern
    df_name = 'bevoelkerung_' + year
    globals()[df_name] = pd.read_csv(file, encoding='ISO-8859-1', delimiter=';')

    # Umwandlung der Spalte "Geschlecht"
    for df in [globals()[df_name]]:
        df['Geschlecht'] = df['Geschlecht'].astype(str)
        df['Geschlecht'] = df['Geschlecht'].replace({'2': 'w', '1': 'm'})

    # Umwandlung der Spalte "Alter"
    for df in [globals()[df_name]]:
        df['Alter'] = df['Alter'].replace({'u3': 'unter 3 Jahren', '3-u6': '3 bis unter 6 Jahren',
                                           '6-u15': '6 bis unter 15 Jahren', '15-u18': '15 bis unter 18 Jahren',
                                           '18-u25': '18 bis unter 25 Jahren', '25-u65': '25 bis unter 65 Jahren'})

    # Entfernung der Zeilen mit den Werten "u65" und "ue65" in der Spalte "Alter"
    for df in [globals()[df_name]]:
        df.drop(df[(df['Alter'] == 'u65') | (df['Alter'] == 'ü65') | (df['Alter'] == 'insg')].index, inplace=True)

    # Spaltennamen aendern
    for df in [globals()[df_name]]:
        df.rename(columns={"Alter": "Altersgruppe", "Anzahl": "Anzahl_Bevoelkerung"}, inplace=True)

In [ ]:
# Tabellenkopf der Bevoelkerungsdaten nach der Umformung anzeigen
bevoelkerung_2021.head()

# 3. Sozial- und Bevoelkerungsdaten zusammenfuegen ("Merge") und Quoten berechnen

## 3.1 Merge

### Plan fuer den Merge (hier fuer 2021, analog auf die uebrigen Jahrgaenge zu uebertragen):

- LB_nach_Altersgruppen_2021 und bevoelkerung_2021:
    - Ziel: Anteil der LB (nach Altersgruppen und Geschlecht) an der jeweiligen Bevoelkerung berechnen
    - Vorbereitung: bevoelkerung_2021 gruppieren nach "AGS" und "Geschlecht", sodass Summe der u65 Jaehrigen gebildet wird fuer SGBII-Quote
    - Merge ueber die Spalten "AGS", "Geschlecht"
- ELB_arbeitsuchend_nach_Berufsausbildung_2021:
    - Ziel: Anteil der arbeitsuchenden ELB nach Berufsausbildung und Geschlecht an der Gesamtzahl der arbeitsuchenden ELB 15-u65 berechnen
    - Kein Merge mit Bevoelkerungsdaten erforderlich
- ELB_arbeitsuchend_nach_Schulabschluessen_2021:
    - Ziel: Anteil der arbeitsuchenden ELB nach Schulabschluss und Geschlecht an der Gesamtzahl der arbeitsuchenden ELB 15-u65 berechnen
    - Kein Merge mit Bevoelkerungsdaten erforderlich

In [ ]:
# Merge der Dataframes fuer 2014
## LB
merged_lb_bev_2014 = LB_nach_Altersgruppen_2014.merge(bevoelkerung_2014, on=["AGS", "Geschlecht", "Altersgruppe"])

# Merge der Dataframes fuer die uebrigen Jahre
## LB
merged_lb_bev_2015 = LB_nach_Altersgruppen_2015.merge(bevoelkerung_2015, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2016 = LB_nach_Altersgruppen_2016.merge(bevoelkerung_2016, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2017 = LB_nach_Altersgruppen_2017.merge(bevoelkerung_2017, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2018 = LB_nach_Altersgruppen_2018.merge(bevoelkerung_2018, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2019 = LB_nach_Altersgruppen_2019.merge(bevoelkerung_2019, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2020 = LB_nach_Altersgruppen_2020.merge(bevoelkerung_2020, on=["AGS", "Geschlecht", "Altersgruppe"])
merged_lb_bev_2021 = LB_nach_Altersgruppen_2021.merge(bevoelkerung_2021, on=["AGS", "Geschlecht", "Altersgruppe"])

In [ ]:
# Tabellenkopf der zusammengefuegten Tabelle fuer 2021 anzeigen
merged_lb_bev_2021.head(4)

## 3.2 Ergebnistabellen stapeln

#### Die Daten liegen aktuell noch in einzelnen Dataframes fuer die jeweiligen Jahrgaenge vor. Wir stapeln diese Dataframes untereinander, um alle Jahrgaenge in einem Dataframe zusammenzufassen.

In [ ]:
stacked_lb_bev = pd.concat([merged_lb_bev_2014, merged_lb_bev_2015, merged_lb_bev_2016, merged_lb_bev_2017, merged_lb_bev_2018, merged_lb_bev_2019, merged_lb_bev_2020, merged_lb_bev_2021], ignore_index=True)
stacked_elb_schul = pd.concat([ELB_arbeitsuchend_nach_Schulabschluessen_2014, ELB_arbeitsuchend_nach_Schulabschluessen_2015, ELB_arbeitsuchend_nach_Schulabschluessen_2016, ELB_arbeitsuchend_nach_Schulabschluessen_2017, ELB_arbeitsuchend_nach_Schulabschluessen_2018, ELB_arbeitsuchend_nach_Schulabschluessen_2019, ELB_arbeitsuchend_nach_Schulabschluessen_2020, ELB_arbeitsuchend_nach_Schulabschluessen_2021], ignore_index=True)
stacked_elb_beruf = pd.concat([ELB_arbeitsuchend_nach_Berufsausbildung_2014, ELB_arbeitsuchend_nach_Berufsausbildung_2015, ELB_arbeitsuchend_nach_Berufsausbildung_2016, ELB_arbeitsuchend_nach_Berufsausbildung_2017, ELB_arbeitsuchend_nach_Berufsausbildung_2018, ELB_arbeitsuchend_nach_Berufsausbildung_2019, ELB_arbeitsuchend_nach_Berufsausbildung_2020, ELB_arbeitsuchend_nach_Berufsausbildung_2021], ignore_index=True)

In [ ]:
# Tabellenkopf des gestapelten Dataframes fuer die LB mit allen Jahrgaengen anzeigen
stacked_lb_bev.head(1)

In [ ]:
stacked_elb_beruf.head(100)

## 3.3 Ergebnistabellen Spalten umbennen

In [ ]:
# Umbenennungen im DataFrame "stacked_lb_bev"
stacked_lb_bev = stacked_lb_bev.rename(columns={"Altersgruppe": "Altersgruppe_LB",
                                                "Anzahl": "Anzahl_LB"})

# Umbenennungen im DataFrame "stacked_elb_schul"
stacked_elb_schul = stacked_elb_schul.rename(columns={"arbeitsuchende ELB insgesamt": "arbeitsuchende_elb_insgesamt"})

# Umbenennungen im DataFrame "stacked_elb_beruf"
stacked_elb_beruf = stacked_elb_beruf.rename(columns={"arbeitsuchende ELB insgesamt": "arbeitsuchende_elb_insgesamt"})

## 3.4 Differenzierung nach Geschlecht aufheben

#### Um die Fallzahlen pro Altersgruppe zu erhoehen, wird die Unterscheidung nach Geschlecht aufgehoben

In [ ]:
grouping_columns = ['AGS', 'Gemeinde', 'Altersgruppe_LB', 'Jahr']

# Gruppieren und Summieren
stacked_lb_bev = stacked_lb_bev.groupby(grouping_columns, as_index=False).agg({
    'Anzahl_LB': 'sum',
    'Anzahl_Bevoelkerung': 'sum'
})

In [ ]:
grouping_columns = ['AGS', 'Gemeinde', 'Schulabschluss', 'Jahr']

# Gruppieren und Summieren
stacked_elb_schul = stacked_elb_schul.groupby(grouping_columns, as_index=False).agg({
    'arbeitsuchende_elb_insgesamt': 'sum',
    'Anzahl': 'sum'
})

In [ ]:
grouping_columns = ['AGS', 'Gemeinde', 'Berufsausbildung', 'Jahr']

# Gruppieren und Summieren
stacked_elb_beruf = stacked_elb_beruf.groupby(grouping_columns, as_index=False).agg({
    'arbeitsuchende_elb_insgesamt': 'sum',
    'Anzahl': 'sum'
})

In [ ]:
# Tabellenkopf nach der Aufhebung der Unterscheidung nach Geschlecht anzeigen
stacked_lb_bev.head()

In [ ]:
stacked_elb_beruf.head()

In [ ]:
stacked_elb_schul.head()

## 3.5 Altersgruppen zusammenfassen, sodass folgende Altersdifferenzierung uebrig bleibt: 0-u6; 6-u15; 15-u18; 18-u65

#### Um die Fallzahlen pro Altersgruppe weiter zu erhoehen, werden einzelne Altersgruppen zusammengefasst

In [ ]:
unique_ages = stacked_lb_bev["Altersgruppe_LB"].unique()

In [ ]:
# Werte in der Spalte 'Altersgruppe_LB' ersetzen, um anschliessend gruppieren zu koennen
replacement_dict = {
    'unter 3 Jahren': '0 bis unter 6 Jahren',
    '3 bis unter 6 Jahren': '0 bis unter 6 Jahren',
    '18 bis unter 25 Jahren': '18 bis unter 65 Jahren',
    '25 bis unter 65 Jahren': '18 bis unter 65 Jahren'
}

stacked_lb_bev['Altersgruppe_LB'] = stacked_lb_bev['Altersgruppe_LB'].replace(replacement_dict)

# Liste der Spalten, nach denen gruppiert werden soll
grouping_columns = ['AGS', 'Gemeinde', 'Altersgruppe_LB', 'Jahr']

# Gruppieren und Summieren
stacked_lb_bev = stacked_lb_bev.groupby(grouping_columns, as_index=False).agg({
    'Anzahl_LB': 'sum',
    'Anzahl_Bevoelkerung': 'sum'
})

In [ ]:
# Tabellenkopf nach Zusammenfassung einzelner Altersgruppen anzeigen
stacked_lb_bev.head()

## 3.6 SGB-II und ELB Quoten berechnen

#### Nachdem die Tabellen mit den absoluten Werten nun in der gewuenschten Form vorliegen, koennen die Quoten berechnet werden

In [ ]:
# Bevoelkerungsdaten enthalten Zeilen, in denen die jeweilige Bevoelkerung = 0 ist; daher Zeilen mit Bevoelkerung = 0 aus Bevoelkerungsdaten entfernen, um Teilen durch 0 bei der Berechnung der Quote zu verhindern
stacked_lb_bev = stacked_lb_bev[stacked_lb_bev.Anzahl_Bevoelkerung != 0]

# Berechnung der SGB2-Quote (%)
stacked_lb_bev["SGB2-Quote (%)"] = (stacked_lb_bev["Anzahl_LB"] / stacked_lb_bev["Anzahl_Bevoelkerung"]) * 100

In [ ]:
# BA-Daten enthalten Zeilen, in denen arbeitsuchende ELB = 0 ist; daher entsprechende Zeilen entfernen, um Teilen durch 0 bei der Berechnung der Quote zu verhindern
stacked_elb_schul = stacked_elb_schul[stacked_elb_schul.arbeitsuchende_elb_insgesamt != 0]

# Berechnung der ELB-Quote nach Schulabschluss (%)
stacked_elb_schul["ELB-Quote (%)"] = (stacked_elb_schul["Anzahl"] / stacked_elb_schul["arbeitsuchende_elb_insgesamt"]) * 100

In [ ]:
# BA-Daten enthalten Zeilen, in denen arbeitsuchende ELB = 0 ist; daher entsprechende Zeilen entfernen, um Teilen durch 0 bei der Berechnung der Quote zu verhindern
stacked_elb_beruf = stacked_elb_beruf[stacked_elb_beruf.arbeitsuchende_elb_insgesamt != 0]

# Berechnung der ELB-Quote nach Berufsausbildung (%)
stacked_elb_beruf["ELB-Quote (%)"] = (stacked_elb_beruf["Anzahl"] / stacked_elb_beruf["arbeitsuchende_elb_insgesamt"]) * 100

In [ ]:
# Tabellenkopf anzeigen
stacked_lb_bev.head()

## 3.7 Problem unplausibler Werte beheben

#### Grund fuer unplausible Werte:

- Fehlerhafte Schaetzung der SGB2-Quote aufgrund unterschiedlicher Datenstruktur:
    - Bei Bevoelkerungsdaten handelt es sich um Stichtagsdaten (31.12.), bei BA-Daten um Jahresdurchschnittswerte
    - Dies kann grundsaetzlich zu einer fehlerhaften Schaetzung der (gruppenspezifischen) SGB2-Quoten fuehren
    - In wenigen Faellen (0,04 %) ist die Anzahl der Leistungsbezieher aufgrund dieser Datenstruktur in einer bestimmten Gruppe (nach Geschlecht, Alter, Jahr, Gemeinde) hoeher als die Anzahl der Bevoelkerung in der jeweiligen Gruppe (z. B. 3 Leistungsbezieher, aber nur eine Person in den Bevoelkerungsdaten)
    - Vorgehen: Ausschluss von Quoten > 100% (SGB2-Quote und ELB-Quote)

In [ ]:
# Faelle entfernen, bei denen SGB2-Quote > 100% ist
stacked_lb_bev = stacked_lb_bev[stacked_lb_bev['SGB2-Quote (%)'] <= 100].reset_index(drop=True)

In [ ]:
# Faelle entfernen, bei denen ELB-Quote > 100% ist
stacked_elb_schul = stacked_elb_schul[stacked_elb_schul['ELB-Quote (%)'] <= 100].reset_index(drop=True)

In [ ]:
# Faelle entfernen, bei denen ELB-Quote > 100% ist
stacked_elb_beruf = stacked_elb_beruf[stacked_elb_beruf['ELB-Quote (%)'] <= 100].reset_index(drop=True)

## 3.8 Datenreihen mit Bezugsgroesse <= 70 entfernen

In [ ]:
# Datensaetze erstellen
stacked_lb_bev_ue70 = stacked_lb_bev[stacked_lb_bev.Anzahl_Bevoelkerung > 70]
stacked_elb_schul_ue70 = stacked_elb_schul[stacked_elb_schul.arbeitsuchende_elb_insgesamt > 70]
stacked_elb_beruf_ue70 = stacked_elb_beruf[stacked_elb_beruf.arbeitsuchende_elb_insgesamt > 70]

In [ ]:
# Berechnen, welcher Anteil der Datenpunkte entfernt wurde
num_rows_stacked_elb_beruf = stacked_elb_beruf.shape[0]
num_rows_stacked_elb_schul = stacked_elb_schul.shape[0]
num_rows_stacked_lb_bev = stacked_lb_bev.shape[0]

num_rows_stacked_elb_beruf_ue70 = stacked_elb_beruf_ue70.shape[0]
num_rows_stacked_elb_schul_ue70 = stacked_elb_schul_ue70.shape[0]
num_rows_stacked_lb_bev_ue70 = stacked_lb_bev_ue70.shape[0]

diff1 = num_rows_stacked_elb_beruf - num_rows_stacked_elb_beruf_ue70
diff2 = num_rows_stacked_elb_schul - num_rows_stacked_elb_schul_ue70
diff3 = num_rows_stacked_lb_bev - num_rows_stacked_lb_bev_ue70

remaining1 = num_rows_stacked_elb_beruf_ue70/num_rows_stacked_elb_beruf*100
remaining2 = num_rows_stacked_elb_schul_ue70/num_rows_stacked_elb_schul*100
remaining3 = num_rows_stacked_lb_bev_ue70/num_rows_stacked_lb_bev*100

print("In der Tabelle 'stacked_elb_beruf' wurden", diff1, "Datenpunkte entfernt. Es sind noch", remaining1, "% der Ausgangswerte enthalten." )
print("In der Tabelle 'stacked_elb_schul' wurden", diff2, "Datenpunkte entfernt. Es sind noch", remaining2, "% der Ausgangswerte enthalten." )
print("In der Tabelle 'stacked_lb_bev' wurden", diff3, "Datenpunkte entfernt. Es sind noch", remaining3, "% der Ausgangswerte enthalten." )

## 3.9 Moegliche Klassen bzw. Container ("bins") fuer die Darstellung der Daten auf eingefaerbter Karte pruefen

__Quantile-Methode__: Bei dieser Methode werden die Bins so gewaehlt, dass jedes Intervall die gleiche Anzahl von Datenpunkten enthaelt. Dies sorgt zwar fuer eine gleichmaessige Verteilung der Farben auf der Karte, allerdings koennen kleine Unterschiede in den Daten zu grossen Unterschieden in den Bins fuehren, wenn die Daten eng beieinander liegen.

In [ ]:
# Anzahl der Bins
num_bins = 5

# Quantile berechnen
quantiles = np.linspace(0, 1, num_bins + 1)
bin_edges = np.quantile(stacked_lb_bev_ue70['SGB2-Quote (%)'], quantiles)

# Histogramm erstellen
plt.hist(stacked_lb_bev_ue70['SGB2-Quote (%)'], bins=bin_edges, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('Quantile-Bins')
plt.show()

# Bins und Anzahl der Werte ausgeben
for i in range(num_bins):
    bin_range = f'{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}'
    count = ((stacked_lb_bev_ue70['SGB2-Quote (%)'] >= bin_edges[i]) & (stacked_lb_bev_ue70['SGB2-Quote (%)'] < bin_edges[i+1])).sum()
    print(f'Bin {i+1}: {bin_range}, Anzahl: {count}')

__Gleichmaessige Intervalle__: Hierbei wird der gesamte Bereich der Daten in gleich grosse Intervalle unterteilt. Diese Einteilung ist leicht verstaendlich, kann jedoch dazu fuehren, dass viele Datenpunkte in ein oder zwei Bins fallen, wenn die Daten stark ungleich verteilt sind.

In [ ]:
# Anzahl der Bins
num_bins = 5

# Gleichmaessige Intervalle berechnen
bin_edges = np.linspace(stacked_lb_bev_ue70['SGB2-Quote (%)'].min(), stacked_lb_bev_ue70['SGB2-Quote (%)'].max(), num_bins + 1)

# Histogramm erstellen
plt.hist(stacked_lb_bev_ue70['SGB2-Quote (%)'], bins=bin_edges, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('Gleichmaessige-Intervalle-Bins')
plt.show()

# Bins und Anzahl der Werte ausgeben
for i in range(num_bins):
    bin_range = f'{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}'
    count = ((stacked_lb_bev_ue70['SGB2-Quote (%)'] >= bin_edges[i]) & (stacked_lb_bev_ue70['SGB2-Quote (%)'] < bin_edges[i+1])).sum()
    print(f'Bin {i+1}: {bin_range}, Anzahl: {count}')

__Standardabweichung__: Hierbei werden Bins basierend auf der statistischen Standardabweichung der Daten erstellt. Dies kann hilfreich sein, um zu sehen, welche Datenpunkte signifikant vom Durchschnitt abweichen.

In [ ]:
# Anzahl der Bins
num_bins = 5

# Standardabweichung berechnen
mean = stacked_lb_bev_ue70['SGB2-Quote (%)'].mean()
std = stacked_lb_bev_ue70['SGB2-Quote (%)'].std()
bin_edges = [mean - 2*std, mean - std, mean, mean + std, mean + 2*std, stacked_lb_bev_ue70['SGB2-Quote (%)'].max()]

# Histogramm erstellen
plt.hist(stacked_lb_bev_ue70['SGB2-Quote (%)'], bins=bin_edges, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('Standardabweichung-Bins')
plt.show()

# Bins und Anzahl der Werte ausgeben
for i in range(len(bin_edges)-1):
    bin_range = f'{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}'
    count = ((stacked_lb_bev_ue70['SGB2-Quote (%)'] >= bin_edges[i]) & (stacked_lb_bev_ue70['SGB2-Quote (%)'] < bin_edges[i+1])).sum()
    print(f'Bin {i+1}: {bin_range}, Anzahl: {count}')

__Benutzerdefinierte Bins__:  Hier kann der Benutzer die Bins manuell festlegen, auf Grundlage seines Wissens ueber die Struktur der Daten, fachlicher Erwaegungen, seiner Anforderungen an die Visualisierung etc.

In [ ]:
# Benutzerdefinierte Bins festlegen
custom_bins = [0, 2, 7, 15, 30, 45, 100]

# Histogramm erstellen
plt.hist(stacked_lb_bev_ue70['SGB2-Quote (%)'], bins=custom_bins, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('Benutzerdefinierte Bins')
plt.show()

# Bins und Anzahl der Werte ausgeben
for i in range(len(custom_bins)-1):
    bin_range = f'{custom_bins[i]}-{custom_bins[i+1]}'
    count = ((stacked_lb_bev_ue70['SGB2-Quote (%)'] >= custom_bins[i]) & (stacked_lb_bev_ue70['SGB2-Quote (%)'] < custom_bins[i+1])).sum()
    print(f'Bin {i+1}: {bin_range}, Anzahl: {count}')

__Natural-Breaks-Bins__: Die Methode Natural Breaks klassifiziert die Datenpunkte in Klassen basierend auf natuerlichen Gruppen in den Daten selbst. Sie versucht, die Varianz innerhalb der Gruppen zu minimieren und die Varianz zwischen den Gruppen zu maximieren.

In [ ]:
# Extrahieren der Werte aus der Spalte 'SGB2-Quote (%)'
data = stacked_lb_bev_ue70['SGB2-Quote (%)'].values

# Berechnen der Bins mit der NaturalBreaks-Methode
classifier = NaturalBreaks(data, k=6)
breaks = classifier.bins
breaks = np.insert(breaks, 0, 0)

# Erstellen des Histogramms
plt.hist(data, bins=breaks, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('Natural-Breaks Bins')
plt.show()

# Ausgabe der Bin-Ranges und der Anzahl der Werte in jedem Bin
for i in range(len(breaks)-1):
    print(f"Bin {i+1}: {breaks[i]}-{breaks[i+1]}, Anzahl: {((data > breaks[i]) & (data <= breaks[i+1])).sum()}")

__FisherJenks_ bzw. FisherJenksSampled__: Aehnlich wie Natural Breaks, jedoch spezialisiert auf die Anwendung auf eindimensionale Daten; sorgt im Vergleich zu aehnlichen Methoden fuer geringere Abweichungen der Datenpunkte um den jeweiligen Klassen-Median. Bei der "Sampled"-Variante (sinnvoll bei sehr grossen Datenmengen) werden nicht alle Datenpunkte, sondern eine Stichprobe (z. B. 20 % der Faelle) zur Ermittlung der Klassengrenzen beruecksichtigt.

In [ ]:
# Extrahieren der Werte aus der Spalte 'SGB2-Quote (%)'
data = stacked_lb_bev_ue70['SGB2-Quote (%)'].values

# Berechnen der Bins mit der FisherJenksSampled-Methode
classifier = FisherJenksSampled(data, pct=0.2, k=6)
bins = classifier.bins
bins = np.insert(bins, 0, 0)

# Erstellen des Histogramms
plt.hist(data, bins=bins, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('FisherJenksSampled Bins')
plt.show()

# Ausgabe der Bin-Ranges und der Anzahl der Werte in jedem Bin
for i in range(len(bins)-1):
    print(f"Bin {i+1}: {bins[i]}-{bins[i+1]}, Anzahl: {((data > bins[i]) & (data <= bins[i+1])).sum()}")

## 3.10 Bins fuer 2021 auf Gemeindeebene mit FisherJenks-Methode ermitteln

In [ ]:
# Daten bereinigen, sodass zusammengefasste SGB2-Quote auf Gemeindeebene fuer das Jahr 2021 verbleibt
data2021_fisher_jenks = stacked_lb_bev_ue70
data2021_fisher_jenks = data2021_fisher_jenks[data2021_fisher_jenks.Jahr == 2021]
data2021_fisher_jenks['LB_Gesamt'] = data2021_fisher_jenks.groupby('Gemeinde')['Anzahl_LB'].transform('sum')
data2021_fisher_jenks['Bev_Gesamt'] = data2021_fisher_jenks.groupby('Gemeinde')['Anzahl_Bevoelkerung'].transform('sum')
data2021_fisher_jenks["SGB2-Quote (%) gesamt"] = (data2021_fisher_jenks["LB_Gesamt"] / data2021_fisher_jenks["Bev_Gesamt"]) * 100
data2021_fisher_jenks = data2021_fisher_jenks[['Gemeinde', 'SGB2-Quote (%) gesamt']]
data2021_fisher_jenks = data2021_fisher_jenks.drop_duplicates()

# Extrahieren der Werte aus der Spalte 'SGB2-Quote (%) gesamt'
data = data2021_fisher_jenks['SGB2-Quote (%) gesamt'].values

# Berechnen der Bins mit der FisherJenks-Methode
classifier = FisherJenks(data, k=5)
bins = classifier.bins
bins = np.insert(bins, 0, 0)

# Erstellen des Histogramms
plt.hist(data, bins=bins, edgecolor='black', color='lightblue', alpha=0.6)
plt.xlabel('SGB2-Quote (%)')
plt.ylabel('Anzahl')
plt.title('FisherJenks Bins')
plt.show()

# Ausgabe der Bin-Ranges und der Anzahl der Werte in jedem Bin
for i in range(len(bins)-1):
    print(f"Bin {i+1}: {bins[i]}-{bins[i+1]}, Anzahl: {((data > bins[i]) & (data <= bins[i+1])).sum()}")

## 3.11 Bins-Spalte ergaenzen

In [ ]:
# Grenzen und Labels fuer die Bins (basierend auf FisherJenks) definieren
bins = [0, 2, 4, 7, 11, 100]
labels = [1, 2, 3, 4, 5]

# Neue 'bins'-Spalte erstellen
stacked_lb_bev_ue70['bins'] = pd.cut(stacked_lb_bev_ue70['SGB2-Quote (%)'], bins=bins, labels=labels)

## 3.12 Problem beheben, das durch 7-stellige AGS bei der Verknuepfung der Daten mit den Shapefiles in tableau entsteht

In [ ]:
# Spalte "AGS" als String definieren
stacked_lb_bev_ue70["AGS"] = stacked_lb_bev_ue70["AGS"].astype(str)

# "0" vor den Eintraegen mit genau 7 Zeichen hinzufuegen
stacked_lb_bev_ue70["AGS"] = stacked_lb_bev_ue70["AGS"].apply(lambda x: '0' + x if len(x) == 7 else x)

## 3.13 AGS fuer Kreisebene ergaenzen (zur Verknuepfung mit den entsprechenden Shapefiles in tableau)

In [ ]:
stacked_lb_bev_ue70['AGS_Kreis'] = stacked_lb_bev_ue70['AGS'].str[:5]

## 3.14 AGS fuer Bundesland ergaenzen (zur Verknuepfung mit den entsprechenden Shapefiles in tableau)

In [ ]:
stacked_lb_bev_ue70['AGS_BL'] = stacked_lb_bev_ue70['AGS'].str[:2]

# 4. Tabelle fuer SBG II-Dynamik-Indikator erstellen

#### Berechnung der Entwicklung der Quote in den Gemeinden in den Jahren 2014 bis 2021

## 4.1 Neues Dataframe erstellen und filtern

In [ ]:
# Neues DataFrame 'sgb_ii_dynamik' als Kopie von 'stacked_lb_bev_ue70' erstellen
sgb_ii_dynamik = stacked_lb_bev_ue70.copy(deep=True)

In [ ]:
# DataFrame 'sgb_ii_dynamik' fuer die Jahre 2014 und 2021 filtern
sgb_ii_dynamik = sgb_ii_dynamik[sgb_ii_dynamik['Jahr'].isin([2014, 2021])]

In [ ]:
# Erste Zeilen des gefilterten DataFrames anzeigen
sgb_ii_dynamik.head(5)

## 4.2 Nicht benoetigte Spalten entfernen

In [ ]:
# Liste der benoetigten Spalten erstellen
gewuenschte_spalten = ["AGS", "Gemeinde", "Altersgruppe_LB", "Jahr", "Anzahl_LB", "Anzahl_Bevoelkerung", "SGB2-Quote (%)", "AGS_Kreis", "AGS_BL"]

# Nur die benoetigten Spalten uebernehmen
sgb_ii_dynamik = sgb_ii_dynamik.loc[:,gewuenschte_spalten]

In [ ]:
# DataFrame anzeigen
sgb_ii_dynamik.head(5)

## 4.3: Dataframe umstrukturieren mit "pivot"

In [ ]:
# unstack mittels pivot-Methode
sgb_ii_dynamik_unstacked = sgb_ii_dynamik.pivot(index=["AGS", "Gemeinde", "Altersgruppe_LB", "AGS_Kreis", "AGS_BL"],
                                                columns="Jahr",
                                                values=["Anzahl_LB", "Anzahl_Bevoelkerung", "SGB2-Quote (%)"])

# Tabellenkopf anzeigen
sgb_ii_dynamik_unstacked.head()

In [ ]:
# Spalten-Index Name entfernen
sgb_ii_dynamik_unstacked.columns.name = None

# Spalten temporaer neu benennen, um Multiindex zu entfernen
sgb_ii_dynamik_unstacked.columns = [f"temp_{col}" for col in sgb_ii_dynamik_unstacked.columns]

In [ ]:
# Index zuruecksetzen
sgb_ii_dynamik_unstacked.reset_index(inplace=True)

In [ ]:
# DataFrame anzeigen
sgb_ii_dynamik_unstacked.head(5)


## 4.4 Spaltennamen final umbenennen

In [ ]:
# Dictionary fuer Umbenennungen der Spalten erstellen
umbenennungen = {
    "temp_('Anzahl_LB', 2014)": "Anzahl_LB_2014",
    "temp_('Anzahl_LB', 2021)": "Anzahl_LB_2021",
    "temp_('Anzahl_Bevoelkerung', 2014)": "Anzahl_Bevoelkerung_2014",
    "temp_('Anzahl_Bevoelkerung', 2021)": "Anzahl_Bevoelkerung_2021",
    "temp_('SGB2-Quote (%)', 2014)": "SGB2-Quote (%)_2014",
    "temp_('SGB2-Quote (%)', 2021)": "SGB2-Quote (%)_2021"
}

In [ ]:
# Umbenennungen auf die Spalten anwenden
sgb_ii_dynamik_unstacked.rename(columns=umbenennungen, inplace=True)

In [ ]:
# DataFrame anzeigen
sgb_ii_dynamik_unstacked.head(5)

## 4.5: Ergebnisse ueberpruefen

In [ ]:
# Anzahl der NaN-Werte in den Spalten "Anzahl_Bevoelkerung" und "Anzahl_LB" berechnen
anzahl_nan_anzahl_bevoelkerung = sgb_ii_dynamik_unstacked['Anzahl_Bevoelkerung_2014'].isna().sum()
anzahl_nan_anzahl_lb = sgb_ii_dynamik_unstacked['Anzahl_LB_2014'].isna().sum()

In [ ]:
# Ergebnisse anzeigen
print(f"Anzahl NaN-Werte Bevoelkerung': {anzahl_nan_anzahl_bevoelkerung}")
print(f"Anzahl NaN-Werte LB': {anzahl_nan_anzahl_lb}")

## 4.6: Fehlersuche und -loesung

In [ ]:
# Anzahl der NaN-Werte im urspruenglichen Dataframe pruefen
anzahl_nan_vor_pivot = stacked_lb_bev_ue70['Anzahl_Bevoelkerung'].isna().sum()
anzahl_nan_vor_pivot

In [ ]:
# Anzahl der NaN-Werte im pivotierten Dataframe pruefen
anzahl_nan_nach_pivot = sgb_ii_dynamik_unstacked['Anzahl_Bevoelkerung_2014'].isna().sum()
anzahl_nan_nach_pivot

In [ ]:
# Stichprobenartige Auswahl von zehn Zeilen, in denen 'Anzahl_Bevoelkerung_2014' oder 'Anzahl_LB_2014' == NaN ist, anzeigen
stichprobe_nan = sgb_ii_dynamik_unstacked[
    sgb_ii_dynamik_unstacked['Anzahl_Bevoelkerung_2014'].isna() |
    sgb_ii_dynamik_unstacked['Anzahl_LB_2014'].isna()
].sample(10)

stichprobe_nan.head

In [ ]:
# Zeilen mit NaN-Werten in den genannten Spalten entfernen
sgb_ii_dynamik_cleaned = sgb_ii_dynamik_unstacked.dropna(subset=['Anzahl_Bevoelkerung_2014', 'Anzahl_LB_2014', 'Anzahl_Bevoelkerung_2021', 'Anzahl_LB_2021'])

In [ ]:
# Bereinigtes DataFrame ueberpruefen
sgb_ii_dynamik_cleaned.info()

In [ ]:
# Fertiges DataFrame umbenennen und die ersten Zeilen anzeigen
sgb_ii_dynamik_final = sgb_ii_dynamik_cleaned
sgb_ii_dynamik_final.head()

## 4.7: Daten aggregieren in Vorbereitung der Berechnung der Quoten und Klassifizierung

In [ ]:
# Daten basierend auf Altersgruppen aggregieren
aggregierte_daten = sgb_ii_dynamik_cleaned.groupby(['AGS', 'Gemeinde', 'AGS_Kreis', 'AGS_BL']).agg({
    'Anzahl_LB_2014': 'sum',
    'Anzahl_LB_2021': 'sum',
    'Anzahl_Bevoelkerung_2014': 'sum',
    'Anzahl_Bevoelkerung_2021': 'sum',
}).reset_index()

In [ ]:
# Aggregierte Tabelle anzeigen
sgb_ii_dynamik_cleaned.head()

## 4.8: Quoten und Dynamik berechnen

In [ ]:
# neue Spalte "SGB2_Quote_2014" erzeugen
aggregierte_daten['SGB2_Quote_2014'] = (aggregierte_daten['Anzahl_LB_2014'] / aggregierte_daten['Anzahl_Bevoelkerung_2014']) * 100

# neue Spalte "SGB2_Quote_2021" erzeugen
aggregierte_daten['SGB2_Quote_2021'] = (aggregierte_daten['Anzahl_LB_2021'] / aggregierte_daten['Anzahl_Bevoelkerung_2021']) * 100

# neue Spalte "Veraenderung" erzeugen
aggregierte_daten['Veraenderung'] = (aggregierte_daten['SGB2_Quote_2021'] - aggregierte_daten['SGB2_Quote_2014'] )

# aggregierte Tabelle mit Veraenderung anzeigen
aggregierte_daten.head()

## 4.9: Intervalle fuer die Darstellung der Dynamik auf der Karte berechnen

In [ ]:
# Mittelwert und Standardabweichung der Werte in der Spalte "Veraenderung" berechnen
mittelwert_veraenderung = aggregierte_daten['Veraenderung'].mean()
standardabweichung_veraenderung = aggregierte_daten['Veraenderung'].std()

In [ ]:
# Ergebnisse anzeigen
print(f"Mittelwert der Veraenderung: {mittelwert_veraenderung}")
print(f"Standardabweichung der Veraenderung: {standardabweichung_veraenderung}")

In [ ]:
# Definition der Intervalle
print(f"Intervall fuer stabile Entwicklung: von {mittelwert_veraenderung - standardabweichung_veraenderung} bis {mittelwert_veraenderung + standardabweichung_veraenderung}")
print(f"Steigende Entwicklung: ab {mittelwert_veraenderung + standardabweichung_veraenderung}")
print(f"Abnehmende Entwicklung: ab {mittelwert_veraenderung - standardabweichung_veraenderung}")


# 5. Fertige Tabellen exportieren als Excel und CSV

In [ ]:
# DataFrame stacked_elb_schul speichern
stacked_elb_schul_ue70.to_excel('elb_quote_schul_ue70.xlsx', index=False)
stacked_elb_schul_ue70.to_csv('elb_quote_schul_ue70.csv', index=False)

# DataFrame stacked_elb_beruf speichern
stacked_elb_beruf_ue70.to_excel('elb_quote_beruf_ue70.xlsx', index=False)
stacked_elb_beruf_ue70.to_csv('elb_quote_beruf_ue70.csv', index=False)

# DataFrame stacked_lb_bev speichern
stacked_lb_bev_ue70.to_excel('sgb2_quote_ue70.xlsx', index=False)
stacked_lb_bev_ue70.to_csv('sgb2_quote_ue70.csv', index=False)

# Dynamik-Indikator speichern
sgb_ii_dynamik_final.to_excel('sgb_ii_dynamik_final.xlsx', index=False)
sgb_ii_dynamik_final.to_csv('sgb_ii_dynamik_final.csv', index=False)

# 6. Diagramme zur Darstellung der Verteilung und Klassifizierung auf der Themenseite erzeugen

## 6.1 Statusindikator

In [ ]:
# Daten vorbereiten
bins = [0, 2, 4, 7, 12, 40] #Rundung der Klassen auf ganze Zahlen
counts = [3197, 3657, 2407, 959, 286] #Die Anzahl der hier angegebenen Gemeinden pro Klasse wird in Tableau berechnet

# Breiten der Bins berechnen
bin_widths = [bins[i+1] - bins[i] for i in range(len(bins)-1)]

# Groesse des Plots anpassen
plt.figure(figsize=(14,8))

# Histogramm erstellen
bars = plt.bar(bins[:-1], counts, width=bin_widths, color='#9AB688', edgecolor='black', align='edge')

# Achsenbezeichnungen festlegen
plt.xlabel('SGB II-Hilfequote')
plt.ylabel('Anzahl der Gemeinden')

# Titel hinzufuegen
plt.title('Verteilung der Gemeinden nach ihrer SGB II-Hilfequote \n(Klassifizierung nach Fisher-Jenks-Algorithmus)',
          fontweight='semibold', pad=30)

# Y-Achse Limit und Ticks einstellen
plt.ylim(0, 4000)
plt.yticks(counts + [4000])

# X-Achse Ticks und Labels einstellen
plt.xticks(bins, [f'{bins[i]}%' for i in range(len(bins))])

# Histogrammbalkenbeschriftung hinzufuegen
for bar, count in zip(bars, counts):
    # Position der Beschriftung in der Mitte des Balkens berechnen
    x = bar.get_x() + bar.get_width() / 2
    y = count

    # Text hinzufuegen
    plt.text(x, y + 200, f"{locale.format_string('%d', count, grouping=True)} \nGem.", color='black', ha='center')

plt.grid(False)

# Rahmenlinien entfernen
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Ticks aktivieren
plt.tick_params(axis='both', which='both', bottom=True, top=False, left=True, right=False)

# Diagramm als Bilddatei speichern
plt.savefig('Histogramm_weiss_balken_gruen.png', dpi=300, bbox_inches='tight')

plt.show()

## 6.2 Dynamikindikator

In [ ]:
# Grenzwerte für die Klasseneinteilung festlegen
start_abnehmend = aggregierte_daten['Veraenderung'].min()
untergrenze_abnehmend = -3.5422641895488765
obergrenze_stabil = 1.093529746625937
obergrenze_steigend = aggregierte_daten['Veraenderung'].max()

# Berechnung der Klassenzugehoerigkeiten
# Gemeinden mit abnehmender Entwicklung
anzahl_abnehmend = (aggregierte_daten['Veraenderung'] < untergrenze_abnehmend).sum()

# Gemeinden mit stabiler Entwicklung
anzahl_stabil = ((aggregierte_daten['Veraenderung'] >= untergrenze_abnehmend) &
                 (aggregierte_daten['Veraenderung'] <= obergrenze_stabil)).sum()

# Gemeinden mit steigender Entwicklung
anzahl_steigend = (aggregierte_daten['Veraenderung'] > obergrenze_stabil).sum()

# Ergebnis der Klasseneinteilung anzeigen
aggregierte_daten_klassifiziert = pd.DataFrame({
    'Klasse': ['Abnehmende Entwicklung', 'Stabile Entwicklung', 'Steigende Entwicklung'],
    'Anzahl': [anzahl_abnehmend, anzahl_stabil, anzahl_steigend]
})

aggregierte_daten_klassifiziert.head()

In [ ]:
# Einstellen des Locale, um Zahlen formatiert auszugeben
locale.setlocale(locale.LC_ALL, 'de_DE')

# Daten vorbereiten
bins = [start_abnehmend, untergrenze_abnehmend, obergrenze_stabil, obergrenze_steigend]  # Endwerte des letzten Balkens anpassen
counts = [anzahl_abnehmend, anzahl_stabil, anzahl_steigend]

# Breiten der Bins berechnen
bin_widths = [bins[i+1] - bins[i] for i in range(len(bins)-1)]

# Groesse des Plots anpassen
plt.figure(figsize=(14,8))

# Histogramm erstellen
bars = plt.bar(bins[:-1], counts, width=bin_widths, color='#9AB688', edgecolor='black', align='edge')

# Achsenbezeichnungen festlegen
plt.xlabel('Veraenderungswerte')
plt.ylabel('Anzahl der Gemeinden')

# Titel hinzufügen
plt.title('Verteilung der Gemeinden nach Veraenderungswerten', fontweight='semibold', pad=30)

# Y-Achse Limit und Ticks einstellen
max_count = max(counts)
plt.ylim(0, max_count + 500)  # Etwas Raum über dem höchsten Balken lassen
plt.yticks(counts + [max_count + 500])

# X-Achse Ticks und Labels einstellen
plt.xticks(bins, [f'{b:.2f}' for b in bins])

# Histogrammbalkenbeschriftung hinzufügen
for bar, count in zip(bars, counts):
    # Position der Beschriftung in der Mitte des Balkens berechnen
    x = bar.get_x() + bar.get_width() / 2
    y = count

    # Text hinzufuegen
    plt.text(x, y + 50, f"{locale.format_string('%d', count, grouping=True)} \nGem.", color='black', ha='center')

plt.grid(False)

# Rahmenlinien entfernen
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Ticks aktivieren
plt.tick_params(axis='both', which='both', bottom=True, top=False, left=True, right=False)

# Diagramm anzeigen
plt.show()